In [7]:

#Denique Black
#Configure

# Quick/dev toggle (True = small/fast run; False = full run per assignment)
QUICK = False
SEED  = 42

# CSV paths (place next to notebook). Also auto-fallback to /mnt/data if present.
NOISE_CSV_PATH        = "noise_matrix_6x6.csv"
DEGRADATION_CSV_PATH  = "degradation_matrix_36x36.csv"

import os
if not os.path.exists(NOISE_CSV_PATH) and os.path.exists("/mnt/data/noise_matrix_6x6.csv"):
    NOISE_CSV_PATH = "/mnt/data/noise_matrix_6x6.csv"
if not os.path.exists(DEGRADATION_CSV_PATH) and os.path.exists("/mnt/data/degradation_matrix_36x36.csv"):
    DEGRADATION_CSV_PATH = "/mnt/data/degradation_matrix_36x36.csv"

# Physical / link parameters (linear units only)
PT     = 0.1    # transmit power (Watts)
L      = 512    # bits per packet
GRID_N = 6      # 6x6 grid → 36 cells

# RL hyperparameters (assignment suggests 100k episodes; we also support early stop)
ALPHA = 0.10
GAMMA = 0.95
EPS_START = 1.00
EPS_END   = 0.05
EPS_DECAY_EPISODES = 40_000

EPISODES             = 100_000 if not QUICK else 2_000
PACKETS_PER_EPISODE  = 500 if not QUICK else 100
STEP_CAP             = 5_000   # safety cap per episode (stop-and-wait retries could be long)

# Early stopping / chunk training (so no waste compute once converged)
CHUNK_EPISODES   = 5_000 if not QUICK else 1_000
MAX_CHUNKS       = max(1, EPISODES // CHUNK_EPISODES)
MA_WINDOW        = 1_000 if not QUICK else 200
PATIENCE_CHUNKS  = 2
MIN_IMPROV_GAIN  = 0.5   # bits/time improvement between chunks to keep going

# Display policy for all transmitters or just a few examples
SHOW_ALL_TX_POLICY = False



# Imports
import math, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
from tqdm import trange

# Reproducibility
rng = np.random.default_rng(SEED)
random.seed(SEED)
np.random.seed(SEED)

print("Using files:")
print("  NOISE_CSV_PATH       =", NOISE_CSV_PATH)
print("  DEGRADATION_CSV_PATH =", DEGRADATION_CSV_PATH)
print("Training config: EPISODES =", EPISODES, "| PACKETS/EP =", PACKETS_PER_EPISODE)


Using files:
  NOISE_CSV_PATH       = noise_matrix_6x6.csv
  DEGRADATION_CSV_PATH = degradation_matrix_36x36.csv
Training config: EPISODES = 100000 | PACKETS/EP = 500


In [8]:

# Load CSVs + Encodings (K=7)

# Load noise (6x6) and pairwise degradation (36x36)
noise_df        = pd.read_csv(NOISE_CSV_PATH, header=None)
degradation_df  = pd.read_csv(DEGRADATION_CSV_PATH, header=None)

assert noise_df.shape == (GRID_N, GRID_N),            f"Expected noise 6x6, got {noise_df.shape}"
assert degradation_df.shape == (GRID_N*GRID_N, GRID_N*GRID_N), f"Expected degradation 36x36, got {degradation_df.shape}"

NOISE = noise_df.values.astype(float)  # Nj at each receiver cell (x,y)
G36   = degradation_df.values.astype(float)  # Gij for flat indices 0..35

def idx_of(x, y, N=GRID_N):    # flatten (x,y) → idx
    return x*N + y

def xy_of(idx, N=GRID_N):      # unflatten idx → (x,y)
    return divmod(idx, N)

def G_of(tx_xy, rx_xy):
    i = idx_of(*tx_xy); j = idx_of(*rx_xy)
    return G36[i, j]

def Nj_of(rx_xy):
    x, y = rx_xy
    return NOISE[x, y]

def pr_of(tx_xy, rx_xy):           # received power
    return PT * G_of(tx_xy, rx_xy)

print("Loaded matrices OK.")
print(" NOISE shape:", NOISE.shape, "| example NOISE[0,0] =", NOISE[0,0])
print(" G36   shape:", G36.shape,   "| example G((0,0)->(5,5)) =", G_of((0,0),(5,5)))

# 7 encodings per assignment (Mk, rk, Dk=rk*log2(Mk))
encodings = pd.DataFrame([
    {"k":1, "Name":"BPSK r=1/2",    "Mk":2,   "rk":0.5},
    {"k":2, "Name":"QPSK r=1/2",    "Mk":4,   "rk":0.5},
    {"k":3, "Name":"8-PSK r=2/3",   "Mk":8,   "rk":2/3},
    {"k":4, "Name":"16-QAM r=3/4",  "Mk":16,  "rk":0.75},
    {"k":5, "Name":"32-QAM r=2/3",  "Mk":32,  "rk":2/3},
    {"k":6, "Name":"64-QAM r=3/4",  "Mk":64,  "rk":0.75},
    {"k":7, "Name":"256-QAM r=3/4", "Mk":256, "rk":0.75},
]).set_index("k").sort_index()
encodings["Dk"] = encodings["rk"] * np.log2(encodings["Mk"])

# Fast lookup
DK = np.zeros(8, dtype=float)           # index 0 unused
DK[1:] = encodings["Dk"].to_numpy()

def Dk_of_fast(k: int) -> float:
    return DK[k]

display(encodings)


Loaded matrices OK.
 NOISE shape: (6, 6) | example NOISE[0,0] = 1.039894e-06
 G36   shape: (36, 36) | example G((0,0)->(5,5)) = 4.866149e-05


,Name,Mk,rk,Dk
k,,,,
1,BPSK r=1/2,2,0.500000,0.500000
2,QPSK r=1/2,4,0.500000,1.000000
3,8-PSK r=2/3,8,0.666667,2.000000
4,16-QAM r=3/4,16,0.750000,3.000000
5,32-QAM r=2/3,32,0.666667,3.333333
6,64-QAM r=3/4,64,0.750000,4.500000
7,256-QAM r=3/4,256,0.750000,6.000000


In [9]:

# Simulator formulas (pr,SNR,
# BER, Psucc) + sanity checks


def SNR_of(tx_xy, rx_xy):
    pr = pr_of(tx_xy, rx_xy)
    Nj = Nj_of(rx_xy)
    return pr / max(Nj, 1e-30)

def BER_of(tx_xy, rx_xy, k: int):
    # BER = exp( - pr / (Nj * Dk) )  with Dk in bits/time-unit
    Dk = Dk_of_fast(k)
    pr = pr_of(tx_xy, rx_xy)
    Nj = Nj_of(rx_xy)
    return math.exp( - pr / (max(Nj, 1e-30) * max(Dk, 1e-30)) )

def Psucc_of(tx_xy, rx_xy, k: int, L_bits=L):
    ber = BER_of(tx_xy, rx_xy, k)
    return (1.0 - ber)**L_bits

# Quick sanity for Link A/B examples from the prompt
def sanity_link(tx, rx, D_list, Nj_override=None, G_override=None):
    pr = pr_of(tx, rx) if G_override is None else PT * G_override
    Nj = Nj_of(rx) if Nj_override is None else Nj_override
    snr = pr / max(Nj, 1e-30)
    print(f"tx={tx} rx={rx} | pr={pr:.3e} W, Nj={Nj:.3e} W, SNR={snr:.3g}")
    for D in D_list:
        ber = math.exp(- snr / D)
        ps  = (1.0 - ber)**L
        print(f"  D={D:>4} → BER={ber:.3e}  Psucc={ps:.3f}")

print("Link A (good): expected ~SNR=50")
sanity_link((2,2),(5,4), D_list=[6.0, 4.5, 1.0], Nj_override=1e-6, G_override=5e-4)
print("\nLink B (weak): expected ~SNR=5")
sanity_link((1,1),(4,1), D_list=[4.5, 1.0, 0.5], Nj_override=1e-6, G_override=5e-5)


Link A (good): expected ~SNR=50
tx=(2, 2) rx=(5, 4) | pr=5.000e-05 W, Nj=1.000e-06 W, SNR=50
  D= 6.0 → BER=2.404e-04  Psucc=0.884
  D= 4.5 → BER=1.495e-05  Psucc=0.992
  D= 1.0 → BER=1.929e-22  Psucc=1.000

Link B (weak): expected ~SNR=5
tx=(1, 1) rx=(4, 1) | pr=5.000e-06 W, Nj=1.000e-06 W, SNR=5
  D= 4.5 → BER=3.292e-01  Psucc=0.000
  D= 1.0 → BER=6.738e-03  Psucc=0.031
  D= 0.5 → BER=4.540e-05  Psucc=0.977


In [10]:

# Environment (Stop&Wait)

class LinkEnv:
    """
    Stop-and-wait link for fixed (tx, rx) during an episode.
    State  = (tx_idx 0..35, rx_idx 0..35, last_enc 0..7; 0 means 'none', last_ack 0/1)
    Action = k in {1..7}
    Reward = throughput-centric:
             r = Dk if packet succeeds, else 0
             (matches assignment's suggested reward; eval metric remains bits/time)
    """
    def __init__(self, tx_xy, rx_xy, packets=PACKETS_PER_EPISODE, step_cap=STEP_CAP):
        self.tx_xy = tuple(tx_xy)
        self.rx_xy = tuple(rx_xy)
        self.packets_total = packets
        self.step_cap = step_cap

        # Cache Psucc for each k
        self.p_succ_k = np.zeros(8, dtype=float)
        for k in range(1, 8):
            self.p_succ_k[k] = Psucc_of(self.tx_xy, self.rx_xy, k)

        self.reset()

    def reset(self):
        self.remaining      = self.packets_total
        self.steps          = 0
        self.delivered_bits = 0
        self.last_enc       = 0
        self.last_ack       = 0
        return self._state()

    def _state(self):
        return (idx_of(*self.tx_xy), idx_of(*self.rx_xy), self.last_enc, self.last_ack)

    def step(self, k: int):
        assert 1 <= k <= 7
        self.steps += 1

        p_succ = self.p_succ_k[k]
        success = (rng.random() < p_succ)

        reward = 0.0
        if success:
            self.delivered_bits += L
            self.remaining -= 1
            self.last_ack = 1
            reward = Dk_of_fast(k)
        else:
            self.last_ack = 0

        self.last_enc = k
        done = (self.remaining <= 0) or (self.steps >= self.step_cap)
        return self._state(), reward, done, {"success": success, "p_succ": p_succ}

    def average_bits_per_time(self):
        return 0.0 if self.steps == 0 else (self.delivered_bits / self.steps)

# Smoke test on a strong link
env = LinkEnv((0,0),(1,1), packets=10, step_cap=200)
_ = env.reset()
for _ in range(20):
    s2, r, done, info = env.step(1)  # BPSK
    if done: break
print("Smoke test avg bits/time:", env.average_bits_per_time())


Smoke test avg bits/time: 512.0


In [11]:

# Q-table + helpers


# Q-table dimensions:
# tx_idx: 0..35 (36) | rx_idx: 0..35 (36) | last_enc: 0..7 (8) | last_ack: 0..1 (2) | action k: 1..7 (we store at 0..6)
Q = np.zeros((GRID_N*GRID_N, GRID_N*GRID_N, 8, 2, 7), dtype=float)

def epsilon_at_episode(ep: int) -> float:
    """Linear epsilon decay from EPS_START to EPS_END."""
    frac = min(1.0, ep / EPS_DECAY_EPISODES)
    return EPS_START + frac * (EPS_END - EPS_START)

def select_action(state, eps: float) -> int:
    """Epsilon-greedy over actions k=1..7."""
    tx_idx, rx_idx, last_enc, last_ack = state
    if rng.random() < eps:
        return int(rng.integers(1, 8))  # explore
    # exploit
    qslice = Q[tx_idx, rx_idx, last_enc, last_ack, :]  # shape (7,)
    return int(np.argmax(qslice) + 1)

def q_update(s, a, r, s2, done: bool):
    """Tabular Q-learning update."""
    tx_i, rx_i, le, la = s
    tx_j, rx_j, le2, la2 = s2
    a_idx = a - 1
    best_next = 0.0 if done else np.max(Q[tx_j, rx_j, le2, la2, :])
    td_target = r + GAMMA * best_next
    td_error  = td_target - Q[tx_i, rx_i, le, la, a_idx]
    Q[tx_i, rx_i, le, la, a_idx] += ALPHA * td_error


In [ ]:

# Training (chunks + ES)


episode_throughputs = []
moving_avg = []
window = deque(maxlen=MA_WINDOW)

best_ma = -1.0
no_improve_chunks = 0
total_eps = 0

for chunk in range(MAX_CHUNKS):
    pbar = trange(CHUNK_EPISODES, desc=f"Chunk {chunk+1}/{MAX_CHUNKS}", leave=False)
    for _ in pbar:
        # Random TX/RX per episode (stationary within the episode)
        tx_xy = (int(rng.integers(0, GRID_N)), int(rng.integers(0, GRID_N)))
        rx_xy = (int(rng.integers(0, GRID_N)), int(rng.integers(0, GRID_N)))

        env = LinkEnv(tx_xy, rx_xy, packets=PACKETS_PER_EPISODE, step_cap=STEP_CAP)
        s = env.reset()
        eps = epsilon_at_episode(total_eps)

        done = False
        while not done:
            a = select_action(s, eps)
            s2, r, done, info = env.step(a)
            q_update(s, a, r, s2, done)
            s = s2

        thr = env.average_bits_per_time()
        episode_throughputs.append(thr)
        window.append(thr)
        ma = float(np.mean(window))
        moving_avg.append(ma)

        total_eps += 1
        if (total_eps % 100) == 0:
            pbar.set_postfix({"MA(bits/time)": f"{ma:.2f}", "ep": total_eps})

    # ---- end of chunk: save & early-stopping check ----
    np.save("Q_checkpoint.npy", Q)
    np.savez(
        "training_metrics.npz",
        episode_throughputs=np.array(episode_throughputs),
        moving_avg=np.array(moving_avg),
        total_eps=total_eps,
    )

    curr_ma = moving_avg[-1]
    gain = curr_ma - best_ma
    print(f"[Chunk {chunk+1}] total_eps={total_eps}, MA(bits/time)={curr_ma:.2f}, gain={gain:.2f}")

    if curr_ma > best_ma + 1e-9:
        # “no improvement” if gain is tiny (< MIN_IMPROV_GAIN)
        if gain < MIN_IMPROV_GAIN:
            no_improve_chunks += 1
        else:
            no_improve_chunks = 0
        best_ma = curr_ma
    else:
        no_improve_chunks += 1

    if no_improve_chunks >= PATIENCE_CHUNKS:
        print("Early stopping: moving average plateaued.")
        break

print("Training done. Last 5 moving averages:", np.round(moving_avg[-5:], 3))


[Chunk 1] total_eps=5000, MA(bits/time)=498.95, gain=499.95


[Chunk 2] total_eps=10000, MA(bits/time)=503.50, gain=4.55


Chunk 3/20:  39%|███▉      | 1960/5000 [00:17<00:32, 94.18it/s, MA(bits/time)=499.64, ep=11900]

In [ ]:
# Plot Learning Curve

plt.figure(figsize=(10,6))
plt.plot(episode_throughputs, alpha=0.4, label="per-episode")
plt.plot(moving_avg, linewidth=2, label=f"moving avg ({MA_WINDOW}-ep window)")
plt.xlabel("Episode")
plt.ylabel("Bits per time unit")
plt.title("Learning Curve: Average Successful Bits per Time Unit vs Episodes")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Display the final learned mapping (i,j) -> k

def policy_grid_for_tx(tx_xy):
    """Return a 6x6 table of the greedy encoding k for this TX to all RX cells,
       using last_enc=0, last_ack=0 as the canonical snapshot."""
    tx_idx = idx_of(*tx_xy, GRID_N)
    grid = np.zeros((GRID_N, GRID_N), dtype=int)
    for rx_x in range(GRID_N):
        for rx_y in range(GRID_N):
            rx_idx = idx_of(rx_x, rx_y, GRID_N)
            qslice = Q[tx_idx, rx_idx, 0, 0, :]   # shape (7,)
            k = int(np.argmax(qslice) + 1)        # actions are 1..7
            grid[rx_x, rx_y] = k
    return pd.DataFrame(grid)

# Print tables
tx_examples = [(0,0), (2,3), (5,5)]
for tx_xy in tx_examples:
    print(f"Encoding policy (i,j) -> k for TX={tx_xy} (rows=rx x, cols=rx y):")
    display(policy_grid_for_tx(tx_xy))



In [ ]:
#  Agent vs Naïve (50 runs, 100 packets)

def greedy_action(state):
    """Greedy action from the learned Q-table (k in 1..7)."""
    tx_idx, rx_idx, last_enc, last_ack = state
    qslice = Q[tx_idx, rx_idx, last_enc, last_ack, :]  # shape (7,)
    return int(np.argmax(qslice) + 1)

def steps_to_deliver_packets(tx_xy, rx_xy, packets, policy_mode="agent"):
    """Simulate until 'packets' delivered; return number of time steps used."""
    env = LinkEnv(tx_xy, rx_xy, packets=packets, step_cap=100_000)
    s = env.reset()
    done = False
    while not done:
        if policy_mode == "agent":
            a = greedy_action(s)
        elif policy_mode == "random":
            a = int(rng.integers(1, 8))
        else:
            raise ValueError("policy_mode must be 'agent' or 'random'")
        s, r, done, info = env.step(a)
    return env.steps

EVAL_RUNS = 50
PACKETS_EVAL = 100

steps_agent  = []
steps_random = []

for _ in range(EVAL_RUNS):
    tx_xy = (int(rng.integers(0, GRID_N)), int(rng.integers(0, GRID_N)))
    rx_xy = (int(rng.integers(0, GRID_N)), int(rng.integers(0, GRID_N)))
    steps_agent.append(steps_to_deliver_packets(tx_xy, rx_xy, PACKETS_EVAL, "agent"))
    steps_random.append(steps_to_deliver_packets(tx_xy, rx_xy, PACKETS_EVAL, "random"))

steps_agent  = np.array(steps_agent)
steps_random = np.array(steps_random)

# Print required numeric comparison
print("=== 100-packet timing over 50 random links ===")
print("Agent  : mean steps =", steps_agent.mean(),  "median =", np.median(steps_agent))
print("Random : mean steps =", steps_random.mean(), "median =", np.median(steps_random))

# Plot required per-run comparison
plt.figure(figsize=(12,5))
x = np.arange(EVAL_RUNS)
w = 0.4
plt.bar(x - w/2, steps_agent,  width=w, label="Agent (greedy)")
plt.bar(x + w/2, steps_random, width=w, label="Naïve (random)")
plt.title(f"Time units to deliver {PACKETS_EVAL} packets (50 random TX/RX pairs)")
plt.xlabel("Run index")
plt.ylabel("Steps (time units)")
plt.legend()
plt.tight_layout()
plt.show()
